In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
import pickle
import json
import time
from pathlib import Path

import torch
import torch.utils.data
from torcheval.metrics.functional import multiclass_confusion_matrix
from sklearn.model_selection import StratifiedKFold
from datasets_utils import  ds_get_info, ds_load, TrainTestDS, TSCDataset, DSInfo, ArtificialProtos
from autoencoder import PermutingConvAutoencoder, train_autoencoder, RegularConvEncoder
from log import create_logger

from typing import Dict

from train_utils import EarlyStopping

from train import EpochType, ProtoTSCoeffs, train_prototsnet, best_stat_saver, get_verbose_logger, BestModelCheckpointer

import pandas as pd
import numpy as np

import traceback

In [3]:
device = torch.device('cuda')
# torch.cuda.set_per_process_memory_fraction(fraction=0.5, device=0)  # or 1, watch out for CUDA_VISIBLE_DEVICES

# datasets should be put in the 'datasets/' directory (downloaded from timeseriesclassification.com)
DATASETS_PATH = Path('datasets')

In [4]:
def experiment_setup(experiment_subpath):
    experiment_dir = Path.cwd() / 'experiments' / experiment_subpath
    os.makedirs(experiment_dir, exist_ok=True)

    shutil.copy(src=Path.cwd()/'autoencoder.py', dst=experiment_dir)
    shutil.copy(src=Path.cwd()/'datasets_utils.py', dst=experiment_dir)
    shutil.copy(src=Path.cwd()/'experiments.ipynb', dst=experiment_dir)
    shutil.copy(src=Path.cwd()/'model.py', dst=experiment_dir)
    shutil.copy(src=Path.cwd()/'push.py', dst=experiment_dir)
    shutil.copy(src=Path.cwd()/'train_utils.py', dst=experiment_dir)
    shutil.copy(src=Path.cwd()/'train.py', dst=experiment_dir)
    
    return experiment_dir

In [9]:
dataset, ds_info = ds_load(DATASETS_PATH, 'Libras', get_info=True)
print(dataset)  # contains dataset.train, dataset.test, and optionally dataset.val, each contain X and y (labels), e.g. dataset.train.X
print(ds_info)

Loading datasets:   0%|          | 0/1 [00:00<?, ?it/s]

TrainTestDS(name='Libras', train=<datasets_utils.TSCDataset object at 0x7f017712d450>, test=<datasets_utils.TSCDataset object at 0x7f0238452050>, val=None)
DSInfo(name='Libras', features=2, ts_len=45, num_classes=15)


In [40]:
# hyperparameters
protos_per_class = 3  # number of prototypes will equal 'protos_per_class * number of classes'
proto_len = 5  # prototype length (number of time steps) - it is latent space length, so due to receptive field in the input space it is longer
proto_features = 32  # number of latent features (dimensions) that input is encoded to
reception = 0.75  # estimate for the fraction of significant features, better to underestimate than overestimate
features_lr = 1e-3  # currently unused, cyclic LR is used, configurable in train.py: ProtoTSNetTrainer._setup_optimizers()
num_warm_epochs = 50  # number of epochs during which encoder weights are frozen, value >0 only makes sense if encoder is pretrained
push_start_epoch = 110  # when to start pushing prototypes onto the input data
push_epochs = range(push_start_epoch, 1000, 30)  # which epochs to push prototypes on
num_last_layer_epochs = 40  # how many epochs to train the last layer (prototypes <-> class mapping)
epochs = 200  # overall number of epochs (PUSH + last layer "epochs" count as one epoch here), set it so that the training ends with PUSH

coeffs = ProtoTSCoeffs(crs_ent=1, l1=1e-3, l1_addon=3e-5)  # how much each element contributes to the loss, l1 is last layer l1 regularization, l1_addon is regularization of feature importance layer

train_batch_size = 32
# reduce in case dataset is small
while train_batch_size > len(dataset.train.X) / 2:
    train_batch_size //= 2
test_batch_size = 128

In [41]:
experiment_name = "LibrasTestExperiment"
curr_experiment_dir = experiment_setup(experiment_name)
log, logclose = create_logger(curr_experiment_dir / "log.txt", display=True)

try:
    params = {
        "protos_per_class": protos_per_class,
        "proto_features": proto_features,
        "proto_len_latent": proto_len,
        "features_lr": features_lr,
        "num_classes": ds_info.num_classes,
        "protos_per_class": protos_per_class,
        "coeffs": coeffs._asdict(),
        "num_warm_epochs": num_warm_epochs,
        "push_start_epoch": push_start_epoch,
        "num_last_layer_epochs": num_last_layer_epochs,
        "epochs": epochs,
    }
    with open(curr_experiment_dir / "params.json", "w") as f:
        json.dump(params, f, indent=4)

    log(
        f"Training for {dataset.name}, proto len {proto_len}, features_lr {features_lr}, protos per class {protos_per_class}, l1_addon {coeffs.l1_addon}",
        flush=True,
        display=True
    )
    log(f'Params: {json.dumps(params, indent=4)}')
    
    whole_training_start = time.time()

    log(f'Training encoder', flush=True, display=True)
    autoencoder = PermutingConvAutoencoder(num_features=ds_info.features, latent_features=proto_features, reception_percent=reception, padding='same')
    train_loader = torch.utils.data.DataLoader(dataset.train, batch_size=train_batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset.val if dataset.val else dataset.test, batch_size=test_batch_size)
    train_autoencoder(autoencoder, train_loader, test_loader, device=device, log=log)
    encoder = autoencoder.encoder

    log(f'Training ProtoTSNet', flush=True, display=True)
    trainer = train_prototsnet(
        dataset,
        curr_experiment_dir,
        device,
        encoder,
        features_lr,
        coeffs,
        protos_per_class,
        proto_features,
        proto_len,
        train_batch_size,
        test_batch_size,
        num_epochs=epochs,
        num_warm_epochs=num_warm_epochs,
        push_start_epoch=push_start_epoch,
        push_epochs=push_epochs,
        ds_info=ds_info,
        num_last_layer_epochs=num_last_layer_epochs,
        custom_checkpointers=[
            get_verbose_logger(dataset.name),
        ],
        log=log,
    )

    accu_test = trainer.latest_stat("accu_test")
    log(f'Last epoch test accu: {accu_test*100:.2f}%', display=True)
    with open(curr_experiment_dir / "test_accu.json", "w") as f:
        json.dump({"value": accu_test}, f, indent=4)
    
    test_loader = torch.utils.data.DataLoader(dataset.test, batch_size=test_batch_size)
    ptsnet = trainer.ptsnet
    confusion_matrix = torch.zeros(ptsnet.num_classes, ptsnet.num_classes)
    for i, (image, label) in enumerate(test_loader):
        output, _ = ptsnet(image.to(device))
        confusion_matrix += multiclass_confusion_matrix(output.to('cpu'), label, num_classes=output.shape[1])
    np.savetxt(curr_experiment_dir / 'confusion_matrix.txt', confusion_matrix.numpy(), fmt='%4d')

    whole_training_end = time.time()
    log(f"Done in {trainer.curr_epoch - 1} epochs, {whole_training_end - whole_training_start:.2f}s", display=True)
except Exception as e:
    log(f"Exception ocurred for {dataset.name}: {e}", display=True)
    tb_str = traceback.format_tb(e.__traceback__)
    log('\n'.join(tb_str), display=True)
    raise
finally:
    logclose()

Training for Libras, proto len 5, features_lr 0.001, protos per class 3, l1_addon 3e-05
Params: {
    "protos_per_class": 3,
    "proto_features": 32,
    "proto_len_latent": 5,
    "features_lr": 0.001,
    "num_classes": 15,
    "coeffs": {
        "crs_ent": 1,
        "clst": 0,
        "sep": 0,
        "l1": 0.001,
        "l1_addon": 3e-05
    },
    "num_warm_epochs": 50,
    "push_start_epoch": 110,
    "num_last_layer_epochs": 40,
    "epochs": 200
}
Training encoder
epoch:   10/300 mse loss: 0.0061
epoch:   20/300 mse loss: 0.0024
epoch:   30/300 mse loss: 0.0018
epoch:   40/300 mse loss: 0.0016
epoch:   50/300 mse loss: 0.0013
epoch:   60/300 mse loss: 0.0014
epoch:   70/300 mse loss: 0.0014
epoch:   80/300 mse loss: 0.0015
epoch:   90/300 mse loss: 0.0017
epoch:  100/300 mse loss: 0.0019
epoch:  110/300 mse loss: 0.0016
epoch:  120/300 mse loss: 0.0017
epoch:  130/300 mse loss: 0.0015
epoch:  140/300 mse loss: 0.0015
epoch:  150/300 mse loss: 0.0015
epoch:  160/300 mse los